# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-27-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-27-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-28 04:58:43,34.223334,-82.461707,272,4,0,268.0,"Abbeville, South Carolina, US",1108.981938,1.470588
1,22001.0,Acadia,Louisiana,US,2020-07-28 04:58:43,30.295065,-92.414197,2215,63,0,2152.0,"Acadia, Louisiana, US",3569.989524,2.844244
2,51001.0,Accomack,Virginia,US,2020-07-28 04:58:43,37.767072,-75.632346,1065,15,0,1050.0,"Accomack, Virginia, US",3295.581136,1.408451
3,16001.0,Ada,Idaho,US,2020-07-28 04:58:43,43.452658,-116.241552,7203,47,0,7156.0,"Ada, Idaho, US",1495.679908,0.652506
4,19001.0,Adair,Iowa,US,2020-07-28 04:58:43,41.330756,-94.471059,20,0,0,20.0,"Adair, Iowa, US",279.642058,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,35301,35475,35526,35615,35727,35928,35981,36036,36157,36263
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,4008,4090,4171,4290,4358,4466,4570,4637,4763,4880
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,22549,23084,23691,24278,24872,25484,26159,26764,27357,27973


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1164,1181,1185,1186,1190,1211,1225,1248,1259,1269
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,111,112,113,117,120,123,128,134,138,144
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1068,1078,1087,1100,1111,1124,1136,1146,1155,1163


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,23273,23634,23741,23741,23924,24550,24602,24793,25180,25198
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2264,2311,2352,2397,2463,2523,2608,2637,2682,2745
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,15744,16051,16400,16646,16983,17369,17369,18076,18088,18837


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,314,314,319,319,330,337,337,337,337,349
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,37,37,38,38,38,38,38,40,40,40


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
513,1023.0,Choctaw,Alabama,US,2020-07-28 04:58:43,32.022273,-88.265644,263,12,0,251.0,"Choctaw, Alabama, US",2089.125427,4.562738
756,1049.0,DeKalb,Alabama,US,2020-07-28 04:58:43,34.459469,-85.807829,1498,9,0,1489.0,"DeKalb, Alabama, US",2094.724036,0.600801
1619,1077.0,Lauderdale,Alabama,US,2020-07-28 04:58:43,34.901719,-87.656247,918,9,0,909.0,"Lauderdale, Alabama, US",989.981559,0.980392


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,4290259,148011,1325804
Brazil,2442375,87618,1846641
India,1480073,33408,951166


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,4290259,148011,1325804,2816444,2020-07-28 04:58:43,37.897967,-91.289447
Brazil,2442375,87618,1846641,508116,2020-07-28 04:58:43,-12.669522,-48.480493
India,1480073,33408,951166,495499,2020-07-28 04:58:43,23.356511,82.081064
Russia,816680,13334,602249,201097,2020-07-28 04:58:43,54.546312,62.120860
South Africa,452529,7067,274925,170537,2020-07-28 04:58:43,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,458121,8494,0
Florida,432747,5931,0
New York,412344,32645,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,458121,8494,0,449627,2020-07-28 04:58:43,37.778248,-120.728666
Florida,432747,5931,0,426816,2020-07-28 04:58:43,28.940755,-82.700744
New York,412344,32645,0,379699,2020-07-28 04:58:43,42.671593,-75.579694
Texas,400336,5713,0,394623,2020-07-28 04:58:43,31.647790,-98.615963
New Jersey,179812,15804,0,164008,2020-07-28 04:58:43,40.286441,-74.622450


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,224051,23500,0
California,Los Angeles,176028,4375,0
Arizona,Maricopa,109988,1807,0
Florida,Miami-Dade,107315,1404,0
Illinois,Cook,103008,4842,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,224051,23500,0,200551,2020-07-28 04:58:43,40.767273,-73.971526,36061.0
California,Los Angeles,176028,4375,0,171653,2020-07-28 04:58:43,34.308284,-118.228241,6037.0
Arizona,Maricopa,109988,1807,0,108181,2020-07-28 04:58:43,33.348359,-112.491815,4013.0
Florida,Miami-Dade,107315,1404,0,105911,2020-07-28 04:58:43,25.611236,-80.551706,12086.0
Illinois,Cook,103008,4842,0,98166,2020-07-28 04:58:43,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,36036,4637,26764,897,916,82,158334,36996,14403,20338,...,300270,1174,19952,14929,417,10306,10,1674,4328,2434
2020-07-26,36157,4763,27357,897,932,82,162526,37317,14935,20472,...,301020,1192,20531,15463,420,10469,10,1681,4481,2512
2020-07-27,36263,4880,27973,907,950,86,167416,37390,15303,20558,...,301708,1202,21209,15988,431,10621,10,1691,4552,2704


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,1248,134,1146,52,39,3,2893,700,155,712,...,45823,34,111,138,0,75,1,474,139,34
2020-07-26,1259,138,1155,52,40,3,2939,705,161,712,...,45837,34,116,142,0,76,1,479,139,34
2020-07-27,1269,144,1163,52,41,3,3059,711,167,713,...,45844,35,121,146,0,78,1,483,140,36


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,24793,2637,18076,803,242,60,68022,26243,9019,18124,...,1427,947,10831,8795,365,3282,8,780,1953,518
2020-07-26,25180,2682,18088,803,242,60,70518,26478,9174,18209,...,1434,948,11105,9746,365,3752,8,797,2350,518
2020-07-27,25198,2745,18837,803,242,65,72575,26665,9311,18246,...,1437,951,11674,9959,365,3752,8,833,2815,542


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20
130,84001097,US,USA,840,1097.0,Mobile,Alabama,US,30.784723,-88.208424,...,5899,6017,6150,6279,6443,6797,6901,7096,7193,7361
109,84001055,US,USA,840,1055.0,Etowah,Alabama,US,34.045673,-86.040519,...,1264,1307,1356,1422,1440,1495,1546,1615,1643,1690
110,84001057,US,USA,840,1057.0,Fayette,Alabama,US,33.720769,-87.738866,...,117,119,121,122,124,128,132,134,138,144
111,84001059,US,USA,840,1059.0,Franklin,Alabama,US,34.442353,-87.842895,...,1011,1020,1027,1038,1044,1051,1064,1075,1078,1098
112,84001061,US,USA,840,1061.0,Geneva,Alabama,US,31.093890,-85.835728,...,134,143,146,150,158,167,178,183,186,192


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-25,78130,2339,0,160041,37981,446452,43771,48776,14175,49,...,90796,390286,37623,1396,352,83609,51849,5775,47870,2446
2020-07-26,79294,2525,0,162014,38623,452288,44318,48776,14290,49,...,93936,394084,37973,1400,352,84567,52635,5889,48827,2475
2020-07-27,81115,2623,0,163827,39447,458121,44547,48983,14406,49,...,96489,400336,38409,1402,364,86072,53321,5999,49417,2520


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-25         900    2620     516  312    569     408    710    1029   
2020-07-26         909    2665     523  317    578     408    714    1076   
2020-07-27         932    2727     529  327    601     409    718    1157   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-25          767      192  ...      98      5       40       26   
2020-07-26          776      194  ...      99      5       41       30   
2020-07-27          779      205  ...     101      5       43       30   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-25            214   275   232          1       45      4  
2020-07-26            215   279   233          0       45      4  
2020-07-27            221   292   235          0       45      5  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-25,1456,20,0,3286,399,8408,1794,4413,579,0,...,964,4990,274,56,7,2075,1494,103,891,25
2020-07-26,1473,20,0,3305,401,8448,1794,4413,579,0,...,967,5076,274,56,7,2078,1501,103,892,25
2020-07-27,1491,21,0,3304,408,8494,1799,4418,579,0,...,978,5713,281,56,7,2082,1518,104,893,25


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-25          20      17       4    2      1      11     34       6   
2020-07-26          20      17       4    2      1      11     34       6   
2020-07-27          20      17       4    2      1      11     35       6   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-25           37        7  ...       0      0        0        0   
2020-07-26           37        7  ...       0      0        0        0   
2020-07-27           38        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-25              0     0     0         24        0      0  
2020-07-26              0     0     0         24        0      0  
2020-07-27              0     0     0         24        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,0.001529,0.014661,0.023128,0.000000,0.040909,0.00000,0.031357,0.010461,0.032473,0.006134,...,0.002571,0.006861,0.030579,0.046694,0.004819,0.021104,0.0,0.000000,0.122407,0.060105
2020-07-26,0.003358,0.027173,0.022157,0.000000,0.017467,0.00000,0.026476,0.008677,0.036937,0.006589,...,0.002498,0.015332,0.029020,0.035769,0.007194,0.015816,0.0,0.004182,0.035351,0.032046
2020-07-27,0.002932,0.024564,0.022517,0.011148,0.019313,0.04878,0.030087,0.001956,0.024640,0.004201,...,0.002286,0.008389,0.033023,0.033952,0.026190,0.014519,0.0,0.005949,0.015845,0.076433


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,0.018776,0.046875,0.008803,0.0,0.114286,0.0,0.030638,0.011561,0.068966,0.001406,...,0.001333,0.000000,0.047170,0.029851,NaN,0.071429,0.0,0.010661,0.022059,0.062500
2020-07-26,0.008814,0.029851,0.007853,0.0,0.025641,0.0,0.015900,0.007143,0.038710,0.000000,...,0.000306,0.000000,0.045045,0.028986,NaN,0.013333,0.0,0.010549,0.000000,0.000000
2020-07-27,0.007943,0.043478,0.006926,0.0,0.025000,0.0,0.040830,0.008511,0.037267,0.001404,...,0.000153,0.029412,0.043103,0.028169,NaN,0.026316,0.0,0.008351,0.007194,0.058824


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,0.007764,0.011120,0.040705,0.0,0.004149,0.000000,0.039345,0.019779,0.009062,0.004545,...,0.001404,0.001057,0.034282,0.082195,0.0,0.206618,0.0,0.001284,0.164580,0.007782
2020-07-26,0.015609,0.017065,0.000664,0.0,0.000000,0.000000,0.036694,0.008955,0.017186,0.004690,...,0.004905,0.001056,0.025298,0.108130,0.0,0.143205,0.0,0.021795,0.203277,0.000000
2020-07-27,0.000715,0.023490,0.041409,0.0,0.000000,0.083333,0.029170,0.007062,0.014934,0.002032,...,0.002092,0.003165,0.051238,0.021855,0.0,0.000000,0.0,0.045169,0.197872,0.046332


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-25,0.027959,0.040018,NaN,0.023928,0.019652,0.014237,0.018831,0.000000,-0.001901,0.0,...,0.019286,0.026086,0.017883,0.007942,0.000000,0.015116,0.020168,0.021581,0.020312,0.017048
2020-07-26,0.014898,0.079521,NaN,0.012328,0.016903,0.013072,0.012497,0.000000,0.008113,0.0,...,0.034583,0.009731,0.009303,0.002865,0.000000,0.011458,0.015159,0.019740,0.019992,0.011856
2020-07-27,0.022965,0.038812,NaN,0.011190,0.021334,0.012897,0.005167,0.004244,0.008118,0.0,...,0.027178,0.015865,0.011482,0.001429,0.034091,0.017797,0.013033,0.018679,0.012083,0.018182


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-25      0.010101  0.059442  0.017751  0.050505  0.030797  0.004926   
2020-07-26      0.010000  0.017176  0.013566  0.016026  0.015817  0.000000   
2020-07-27      0.025303  0.023265  0.011472  0.031546  0.039792  0.002451   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-07-25      0.011396  0.060825  0.010540  0.005236  ...  0.010309    0.0   
2020-07-26      0.005634  0.045675  0.011734  0.010417  ...  0.010204    0.0   
2020-07-27      0.005602  0.075279  0.003866  0.056701  ...  0.020202    0.0   

Province_State                                                               \
Admin2         Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-25      0.00000  0.040000   0.019048  0.041667  0.000000        inf   
2020-07-26      0.02500  0.153846   0.004673  0.014545  0.004310       -1.0   
2020-07-27      0.04878  0.000000   0.027907  0.046595  0.008584        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-07-25          0.0   0.00  
2020-07-26          0.0   0.00  
2020-07-27          0.0   0.25  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-25,0.012517,0.052632,NaN,0.045498,0.012690,0.008516,0.002235,0.000000,0.00173,NaN,...,0.027719,0.038718,0.003663,0.0,0.0,0.003870,-0.000669,0.000000,0.014806,0.0
2020-07-26,0.011676,0.000000,NaN,0.005782,0.005013,0.004757,0.000000,0.000000,0.00000,NaN,...,0.003112,0.017234,0.000000,0.0,0.0,0.001446,0.004685,0.000000,0.001122,0.0
2020-07-27,0.012220,0.050000,NaN,-0.000303,0.017456,0.005445,0.002787,0.001133,0.00000,NaN,...,0.011375,0.125493,0.025547,0.0,0.0,0.001925,0.011326,0.009709,0.001121,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                        \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock    Butler Calhoun   
2020-07-25         0.0     0.0     0.0  0.0    0.0     0.0  0.000000     0.0   
2020-07-26         0.0     0.0     0.0  0.0    0.0     0.0  0.000000     0.0   
2020-07-27         0.0     0.0     0.0  0.0    0.0     0.0  0.029412     0.0   

Province_State                     ... Wyoming                           \
Admin2          Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-25      0.000000      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-26      0.000000      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-27      0.027027      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-25            NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-26            NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-27            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,0.002197,0.018782,0.024376,0.002700,0.040562,0.019790,0.035300,0.011262,0.029533,0.006176,...,0.002491,0.014027,0.028344,0.043669,0.008008,0.027757,6.467518e-12,0.005027,0.080005,0.067337
2020-07-26,0.002778,0.022978,0.023266,0.001350,0.029014,0.009895,0.030888,0.009969,0.033235,0.006383,...,0.002494,0.014679,0.028682,0.039719,0.007601,0.021787,3.233759e-12,0.004604,0.057678,0.049691
2020-07-27,0.002855,0.023771,0.022892,0.006249,0.024164,0.029338,0.030488,0.005963,0.028938,0.005292,...,0.002390,0.011534,0.030853,0.036836,0.016896,0.018153,1.616879e-12,0.005277,0.036761,0.063062


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,0.015055,0.039908,0.009807,1.783323e-14,0.079756,1.987560e-31,0.035664,0.011661,0.054504,0.000750,...,0.001690,0.002051,0.042282,0.033760,NaN,0.052633,0.0,0.011562,0.025172,0.077637
2020-07-26,0.011935,0.034879,0.008830,8.916615e-15,0.052698,9.937799e-32,0.025782,0.009402,0.046607,0.000375,...,0.000998,0.001025,0.043664,0.031373,NaN,0.032983,0.0,0.011055,0.012586,0.038819
2020-07-27,0.009939,0.039179,0.007878,4.458307e-15,0.038849,4.968899e-32,0.033306,0.008956,0.041937,0.000890,...,0.000575,0.015219,0.043384,0.029771,NaN,0.029649,0.0,0.009703,0.009890,0.048821


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-25,0.008447,0.019986,0.025590,2.864006e-08,0.017687,0.009728,0.039493,0.019950,0.011645,0.005034,...,0.001648,0.003455,0.031818,0.074810,0.000587,0.155830,3.789561e-14,0.010293,0.086265,0.010549
2020-07-26,0.012028,0.018525,0.013127,1.432003e-08,0.008844,0.004864,0.038094,0.014452,0.014415,0.004862,...,0.003277,0.002256,0.028558,0.091470,0.000293,0.149518,1.894781e-14,0.016044,0.144771,0.005274
2020-07-27,0.006371,0.021008,0.027268,7.160014e-09,0.004422,0.044099,0.033632,0.010757,0.014674,0.003447,...,0.002684,0.002710,0.039898,0.056663,0.000147,0.074759,9.473903e-15,0.030607,0.171322,0.025803


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-25,0.027066,0.035455,NaN,0.021436,0.022659,0.018872,0.016891,0.003098,0.005971,1.250526e-40,...,0.022974,0.024957,0.018752,0.006791,0.023150,0.013939,0.018086,0.027428,0.021051,0.020720
2020-07-26,0.020982,0.057488,NaN,0.016882,0.019781,0.015972,0.014694,0.001549,0.007042,6.252630e-41,...,0.028779,0.017344,0.014027,0.004828,0.011575,0.012699,0.016623,0.023584,0.020521,0.016288
2020-07-27,0.021974,0.048150,NaN,0.014036,0.020558,0.014434,0.009931,0.002896,0.007580,3.126315e-41,...,0.027978,0.016604,0.012755,0.003128,0.022833,0.015248,0.014828,0.021132,0.016302,0.017235


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-25      0.013327  0.055478  0.016759  0.039866  0.039261  0.005832   
2020-07-26      0.011663  0.036327  0.015162  0.027946  0.027539  0.002916   
2020-07-27      0.018483  0.029796  0.013317  0.029746  0.033666  0.002683   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-25      0.008976  0.057597  0.008453  0.023485  ...  0.015295   
2020-07-26      0.007305  0.051636  0.010094  0.016951  ...  0.012749   
2020-07-27      0.006454  0.063458  0.006980  0.036826  ...  0.016476   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-07-25      0.015625  0.000569  0.120281   0.023101  0.045919  0.007775   
2020-07-26      0.007813  0.012784  0.137064   0.013887  0.030232  0.006043   
2020-07-27      0.003906  0.030782  0.068532   0.020897  0.038414  0.007313   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-07-25            NaN  0.001603  0.001546  
2020-07-26           -1.0  0.000801  0.000773  
2020-07-27           -1.0  0.000401  0.125386  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-25,0.020057,0.030018,NaN,0.036294,0.015857,0.012766,0.003175,0.000326,0.024872,NaN,...,0.024999,0.038725,0.013994,6.614510e-14,0.010417,0.004134,0.003563,0.004037,0.010304,0.001103
2020-07-26,0.015866,0.015009,NaN,0.021038,0.010435,0.008762,0.001588,0.000163,0.012436,NaN,...,0.014056,0.027979,0.006997,3.307255e-14,0.005208,0.002790,0.004124,0.002019,0.005713,0.000552
2020-07-27,0.014043,0.032504,NaN,0.010368,0.013946,0.007104,0.002187,0.000648,0.006218,NaN,...,0.012716,0.076736,0.016272,1.653628e-14,0.002604,0.002357,0.007725,0.005864,0.003417,0.000276


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                 \
Admin2           Autauga   Baldwin   Barbour      Bibb Blount   Bullock   
2020-07-25      0.000224  0.026619  0.005331  0.000244    0.0  0.006250   
2020-07-26      0.000112  0.013310  0.002665  0.000122    0.0  0.003125   
2020-07-27      0.000056  0.006655  0.001333  0.000061    0.0  0.001563   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...    Park   
2020-07-25      0.004845  0.000049  0.024098  2.087225e-11  ...     NaN   
2020-07-26      0.002422  0.000024  0.012049  1.043613e-11  ...     NaN   
2020-07-27      0.015917  0.000012  0.019538  5.218063e-12  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-07-25        NaN      NaN      NaN        NaN   NaN   NaN   0.001153   
2020-07-26        NaN      NaN      NaN        NaN   NaN   NaN   0.000576   
2020-07-27        NaN      NaN      NaN        NaN   NaN   NaN   0.000288   

Province_State                  
Admin2         Washakie Weston  
2020-07-25          NaN    NaN  
2020-07-26          NaN    NaN  
2020-07-27          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200727,AK,3206,204162,NaN,38.0,NaN,NaN,NaN,3.0,...,207368,1,0,de28ff6d7d2874c791219904c2fa81f83142a78f,0,0,0,0,0,NaN
1,20200727,AL,81115,576844,NaN,1599.0,9694.0,NaN,1081.0,NaN,...,657959,18,537,906fddfa88864f88bdf0d2bcb19cf267a406f2e6,0,0,0,0,0,NaN
2,20200727,AR,39447,445444,NaN,489.0,2624.0,NaN,NaN,110.0,...,484891,7,82,819db85fb84f50809d492ae373ea4914e01077f0,0,0,0,0,0,NaN
3,20200727,AS,0,1037,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,e2900aeb060df366a3f3a70300dccec3b2e33ae8,0,0,0,0,0,NaN
4,20200727,AZ,163827,714506,NaN,2626.0,7751.0,820.0,NaN,567.0,...,878333,-1,45,11d6f676a2f3a2381d94968e48b3b0b447284a05,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200727,AK,3206.0,204162.0,NaN,38.0,NaN,NaN,NaN,3.0,...,207368,1,0,de28ff6d7d2874c791219904c2fa81f83142a78f,0,0,0,0,0,NaN
1,20200727,AL,81115.0,576844.0,NaN,1599.0,9694.0,NaN,1081.0,NaN,...,657959,18,537,906fddfa88864f88bdf0d2bcb19cf267a406f2e6,0,0,0,0,0,NaN
2,20200727,AR,39447.0,445444.0,NaN,489.0,2624.0,NaN,NaN,110.0,...,484891,7,82,819db85fb84f50809d492ae373ea4914e01077f0,0,0,0,0,0,NaN
3,20200727,AS,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,e2900aeb060df366a3f3a70300dccec3b2e33ae8,0,0,0,0,0,NaN
4,20200727,AZ,163827.0,714506.0,NaN,2626.0,7751.0,820.0,NaN,567.0,...,878333,-1,45,11d6f676a2f3a2381d94968e48b3b0b447284a05,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-27,AK,3206,204162,NaN,38.0,NaN,NaN,NaN,3.0,NaN,...,207368,1,0,de28ff6d7d2874c791219904c2fa81f83142a78f,0,0,0,0,0,NaN
2020-07-27,AL,81115,576844,NaN,1599.0,9694.0,NaN,1081.0,NaN,577.0,...,657959,18,537,906fddfa88864f88bdf0d2bcb19cf267a406f2e6,0,0,0,0,0,NaN
2020-07-27,AR,39447,445444,NaN,489.0,2624.0,NaN,NaN,110.0,369.0,...,484891,7,82,819db85fb84f50809d492ae373ea4914e01077f0,0,0,0,0,0,NaN
2020-07-27,AS,0,1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,e2900aeb060df366a3f3a70300dccec3b2e33ae8,0,0,0,0,0,NaN
2020-07-27,AZ,163827,714506,NaN,2626.0,7751.0,820.0,NaN,567.0,NaN,...,878333,-1,45,11d6f676a2f3a2381d94968e48b3b0b447284a05,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-27,AK,3206.0,204162.0,NaN,38.0,NaN,NaN,NaN,3.0,NaN,...,207368,1,0,de28ff6d7d2874c791219904c2fa81f83142a78f,0,0,0,0,0,NaN
2020-07-27,AL,81115.0,576844.0,NaN,1599.0,9694.0,NaN,1081.0,NaN,577.0,...,657959,18,537,906fddfa88864f88bdf0d2bcb19cf267a406f2e6,0,0,0,0,0,NaN
2020-07-27,AR,39447.0,445444.0,NaN,489.0,2624.0,NaN,NaN,110.0,369.0,...,484891,7,82,819db85fb84f50809d492ae373ea4914e01077f0,0,0,0,0,0,NaN
2020-07-27,AS,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,e2900aeb060df366a3f3a70300dccec3b2e33ae8,0,0,0,0,0,NaN
2020-07-27,AZ,163827.0,714506.0,NaN,2626.0,7751.0,820.0,NaN,567.0,NaN,...,878333,-1,45,11d6f676a2f3a2381d94968e48b3b0b447284a05,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE